In [1]:
import sys
import os
# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import yaml
from datetime import datetime as dt, date
import requests
import logging
from datetime import datetime, timedelta
from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut
from src.databaseService.Parser import Parser_AV
from src.databaseService.CleanData import CleanData
from src.common.AssetDataService import AssetDataService

from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData

import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter(fmt="%(message)s")
handler.setFormatter(formatter)
if not logger.hasHandlers():
    logger.addHandler(handler)
else:
    logger.handlers[:] = [handler]
# Usage
logger.info("This will print to the notebook’s output cell")

This will print to the notebook’s output cell


In [2]:
# Define paths
current_dir = os.getcwd()
desired_folder = "secrets"
absolute_path_to_folder = os.path.join(os.path.abspath(os.path.join(current_dir, "..")), "secrets")

# Path to the YAML file
yaml_file_path = os.path.join("../secrets", "alphaVantage.yaml")

# Read and load the YAML file
try:
    with open(yaml_file_path, 'r') as file:  # Open the YAML file for reading
        config = yaml.safe_load(file)  # Load the YAML content
        apiKey = config['alphaVantage_premium']['apiKey']  # Access the required key
except PermissionError:
    print("Permission denied. Please check file permissions.")
except FileNotFoundError:
    print("File not found. Please verify the path.")
except KeyError:
    print("KeyError: Check the structure of the YAML file.")
except yaml.YAMLError as e:
    print("YAML Error:", e)

In [3]:
ticker = "WBD"
ts = TimeSeries(key=apiKey, output_format='pandas')
fd = FundamentalData(key=apiKey, output_format='pandas')

datashareprice, _ = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
loadad_overview, _ = fd.get_company_overview(symbol=ticker)

url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol='+ticker+'&apikey='+apiKey
incStatementData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol='+ticker+'&apikey='+apiKey
cashFlowData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol='+ticker+'&apikey='+apiKey
balanceSheetData = requests.get(url).json()
url = 'https://www.alphavantage.co/query?function=EARNINGS&symbol='+ticker+'&apikey='+apiKey
earningsData = requests.get(url).json()

if incStatementData=={} or cashFlowData == {} or balanceSheetData == {} or earningsData == {}:
    raise ImportError(f"Empty Financial Data")

In [4]:
res = Parser_AV(sharepriceData=datashareprice).parse_shareprice()

res.tail(10)

,Date,Open,High,Low,Close,AdjClose,Volume,Dividends,Splits
4984,2025-05-02,8.52,8.67,8.43,8.54,8.54,31725510,0.0,1
4985,2025-05-05,8.175,8.53,8.075,8.37,8.37,42739687,0.0,1
4986,2025-05-06,8.182,8.46,8.13,8.43,8.43,38438715,0.0,1
4987,2025-05-07,8.5,8.65,8.4349,8.56,8.56,52292332,0.0,1
4988,2025-05-08,8.16,9.3,8.06,9.01,9.01,69402576,0.0,1
4989,2025-05-09,9.01,9.29,8.91,9.07,9.07,30000498,0.0,1
4990,2025-05-12,9.55,9.785,9.2,9.23,9.23,47028760,0.0,1
4991,2025-05-13,9.28,9.345,9.14,9.18,9.18,35665551,0.0,1
4992,2025-05-14,9.12,9.21,9.02,9.21,9.21,32979575,0.0,1
4993,2025-05-15,9.11,9.25,9.05,9.14,9.14,24005528,0.0,1


In [5]:
loadad_about, loaded_sector = Parser_AV(overview=loadad_overview).parse_overview()

print("About: ", loadad_about)
print("Sector: ", loaded_sector)

About:  {'Symbol': 'WBD', 'AssetType': 'Common Stock', 'Name': 'Warner Bros Discovery Inc', 'Description': 'Warner Bros. The company is headquartered in New York, New York.', 'CIK': '1437107', 'Exchange': 'NASDAQ', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'TECHNOLOGY', 'Industry': 'CABLE & OTHER PAY TELEVISION SERVICES', 'Address': '8403 COLESVILLE ROAD, SILVER SPRING, MD, US', 'OfficialSite': 'https://ir.wbd.com', 'FiscalYearEnd': 'December', 'LatestQuarter': '2025-03-31', 'MarketCapitalization': '22613000000', 'EBITDA': '7585000000', 'PERatio': 'None', 'PEGRatio': '2.786', 'BookValue': '13.68', 'DividendPerShare': 'None', 'DividendYield': 'None', 'EPS': '-4.4', 'RevenuePerShareTTM': '15.62', 'ProfitMargin': '-0.282', 'OperatingMarginTTM': '0.0246', 'ReturnOnAssetsTTM': '0.005', 'ReturnOnEquityTTM': '-0.273', 'RevenueTTM': '38342001000', 'GrossProfitTTM': '16601000000', 'DilutedEPSTTM': '-4.4', 'QuarterlyEarningsGrowthYOY': '2.267', 'QuarterlyRevenueGrowthYOY': '-0.098', 'Analys

In [6]:
financials_annually, financials_quarterly = Parser_AV(
    incStatementData = incStatementData, 
    cashFlowData = cashFlowData, 
    balanceSheetData = balanceSheetData, 
    earningsData = earningsData
).parse_financials()

In [7]:
financials_annually.tail(10)

,fiscalDateEnding,reportedEPS,grossProfit,totalRevenue,ebit,ebitda,totalAssets,totalCurrentLiabilities,totalShareholderEquity,operatingCashflow
10,2015-12-31,1.58,4.051000e+09,6.394000e+09,1.720000e+08,2.211000e+09,1.586400e+10,1.579000e+09,5.451000e+09,1.277000e+09
11,2016-12-31,1.96,4.065000e+09,6.497000e+09,2.021000e+09,2.343000e+09,1.575800e+10,1.561000e+09,5.167000e+09,1.373000e+09
12,2017-12-31,1.91,4.217000e+09,6.873000e+09,2.230000e+09,2.560000e+09,2.255500e+10,1.871000e+09,4.610000e+09,1.629000e+09
13,2018-12-31,2.45,6.618000e+09,1.055300e+10,1.765000e+09,3.163000e+09,3.255000e+10,3.997000e+09,8.386000e+09,2.576000e+09
14,2019-12-31,3.69,7.325000e+09,1.114400e+10,3.013000e+09,4.360000e+09,3.373500e+10,3.239000e+09,9.891000e+09,3.399000e+09
15,2020-12-31,3.21,6.811000e+09,1.067100e+10,2.420000e+09,3.779000e+09,3.408700e+10,3.082000e+09,1.046400e+10,2.739000e+09
16,2021-12-31,1.74,7.571000e+09,1.219100e+10,2.073000e+09,3.655000e+09,3.442700e+10,3.459000e+09,1.159900e+10,2.798000e+09
17,2022-12-31,-0.30,1.337500e+10,3.381700e+10,-7.152000e+09,4.100000e+07,1.340010e+11,1.501700e+10,4.709500e+10,4.304000e+09
18,2023-12-31,-1.26,1.679500e+10,4.132100e+10,-1.601000e+09,6.384000e+09,1.227570e+11,1.533200e+10,4.522600e+10,7.477000e+09
19,2024-12-31,-0.75,1.635100e+10,3.932100e+10,-1.340500e+10,-6.368000e+09,1.045600e+11,1.581000e+10,3.403700e+10,5.375000e+09


In [8]:
financials_quarterly.tail(10)

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime,grossProfit,totalRevenue,ebit,ebitda,totalAssets,totalCurrentLiabilities,totalShareholderEquity,commonStockSharesOutstanding,operatingCashflow
71,2022-12-31,2023-02-23,-0.86,-0.2100,-0.6500,-309.5238,post-market,4.054000e+09,1.100800e+10,-1.978000e+09,1.910000e+08,1.340010e+11,1.501700e+10,4.709500e+10,2.429000e+09,2.846000e+09
72,2023-03-31,2023-05-05,-0.44,0.0100,-0.4500,-4500.0000,pre-market,4.015000e+09,1.070000e+10,-6.610000e+08,1.397000e+09,1.305840e+11,1.638000e+10,4.649600e+10,2.432000e+09,-6.310000e+08
73,2023-06-30,2023-08-03,-0.49,-0.3800,-0.1100,-28.9474,pre-market,3.722000e+09,1.035800e+10,-9.010000e+08,1.013000e+09,1.286180e+11,1.690600e+10,4.545200e+10,2.437000e+09,2.014000e+09
74,2023-09-30,2023-11-08,-0.17,-0.0600,-0.1100,-183.3333,post-market,4.670000e+09,9.979000e+09,4.800000e+07,2.037000e+09,1.237490e+11,1.458800e+10,4.477400e+10,2.438000e+09,2.516000e+09
75,2023-12-31,2024-02-23,-0.16,-0.0700,-0.0900,-128.5714,pre-market,2.361000e+09,1.028900e+10,-8.800000e+07,1.936000e+09,1.227570e+11,1.533200e+10,4.522600e+10,2.436000e+09,3.578000e+09
76,2024-03-31,2024-05-09,-0.24,-0.4000,0.1600,40.0000,pre-market,1.997000e+09,9.954000e+09,-2.990000e+08,1.589000e+09,1.198190e+11,1.695600e+10,4.415100e+10,2.443000e+09,5.850000e+08
77,2024-06-30,2024-08-07,-0.36,-0.2200,-0.1400,-63.6364,post-market,1.767000e+09,9.711000e+09,-1.055300e+10,-8.809000e+09,1.080290e+11,1.776800e+10,3.434500e+10,2.451000e+09,1.228000e+09
78,2024-09-30,2024-11-07,0.05,-0.0800,0.1300,162.5000,pre-market,4.442000e+09,9.623000e+09,-6.720000e+08,1.090000e+09,1.063330e+11,1.569500e+10,3.510000e+10,2.470000e+09,1.453000e+09
79,2024-12-31,2025-02-21,-0.20,0.0257,-0.2257,-878.2101,pre-market,8.139000e+09,1.002700e+10,1.340000e+08,4.844000e+09,1.045600e+11,1.581000e+10,3.403700e+10,2.454000e+09,2.715000e+09
80,2025-03-31,2025-05-07,-0.18,-0.1295,-0.0505,-38.9961,pre-market,3.848000e+09,8.979000e+09,3.400000e+07,4.726000e+09,1.016790e+11,1.528600e+10,3.383600e+10,2.462000e+09,5.530000e+08
